Within `animalDataPath`, must have folder 'NoRMCorred' with motion corrected .tif files

In [1]:
import os
from scipy.io import loadmat
import numpy as np
import fissa
import re
import glob

In [3]:
animalDataPath = r'C:\Data\Manoj Data\Region01'

In [ ]:
os.chdir(animalDataPath)
try:
    anml = re.search(r'[A-Z]{2}\d{4}',animalDataPath)[0]
except:
    anml = "test"
tiff_folder = os.path.join(animalDataPath,'NoRMCorred')
if not os.path.isdir(tiff_folder):
    os.mkdir(tiff_folder)

output_folder = os.path.join(tiff_folder,'FISSAoutput')
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

In [5]:
ROIcoor, nTif, rois_FISSA = [[] for _ in range(3)]
#[::-1] reverses list so pre comes before post
for fileN,ROIfile in enumerate(glob.glob(animalDataPath + '\\*_moCorrROI_*.mat')[::-1]):
    ROIcoor.append(loadmat(ROIfile)['moCorROI']['ROIcurveOrderedXY'].transpose())
    nTif.append(loadmat(ROIfile)['nTifs'][0][0])
    rois_FISSA.extend([[[ROIcoor[fileN][i,0][0],\
                        ROIcoor[fileN][i,0][1]] for i in range(ROIcoor[fileN].shape[0])]]*nTif[fileN])

In [6]:
#This checks whether ROI files all have the same number of ROI:
#this works for however long ROIcorr is:
if not all([coor.shape[0] is ROIcoor[0].shape[0] for coor in ROIcoor]):
    raise NameError('Different number of ROI in some set')

In [7]:
#save ROIs in python format | can write code to split using nTif variable
#rois_FISSA is a list of ROIs of length number of tifs
np.save(os.path.join(tiff_folder,'FISSA_ROIs.npy'),rois_FISSA)

c:\Codes\fissa_process\env_fissa_conda_min\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
print(len(rois_FISSA))
print(len(rois_FISSA[0]))
print(len(rois_FISSA[0][0]))
print(len(rois_FISSA[0][0][0]))
print(len(rois_FISSA[0][0][1]))

for el in rois_FISSA:
    print('\n')
    print(len(el))
    print(len(el[0]))
    print(len(el[0][0]))
    print(len(el[0][1]))

10
74
2
40
40


In [10]:
exp = fissa.Experiment(tiff_folder,rois_FISSA,output_folder)

In [11]:
#run FISSA
exp.separate(redo_prep=True)

#save FISSA output to matlab format
os.chdir(output_folder)
# exp.save_to_matlab()
print('DONE!')

Doing region growing and data extraction....
Doing signal separation....
DONE!
